In [1]:
%load_ext autoreload
%autoreload 2
from models.factory.ModelFactory import ModelFactory
#from evaluation.VQA_Evaluator import VQA_Evaluator
from dataset.VQAv2Dataset import VQAv2Dataset
from torch.utils.data import DataLoader
#import json
#import os
import torch
import tqdm

In [2]:
model_factory = ModelFactory()
model_type = 'murel'
config = model_factory.create_config(model_type)
wid_to_word = torch.load('/auto/homes/bat34/VQA_PartII/data/processed_splits/wid_to_word.pth')
word_vocabulary = [wid_to_word[i] for i in range(1, len(wid_to_word) + 1)]
model = model_factory.create_model(model_type, config, word_vocabulary)
#OLD_TRAINED_FILES = '/local/scratch/bat34/baseline_trained_models/best_models/{}/best_model.pth'
OLD_TRAINED_FILES = '/local/scratch/bat34/murel_trained_models/best_models/{}/best_model.pth'
#MODEL_NAME = 'agg_concat_sum_corrected_wid_seed_1337_agg_type_sum_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0003_fusion_type_concat_mlp'
#MODEL_NAME = 'concat_baseline_self_attention_corrected_wid_seed_2048_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0003_lr_decay_rate_0.25_hidden_list_[1600, 1600]_attention_fusion_type_block_final_fusion_type_block'
#MODEL_NAME = 'graph_conv_graph_type_knn24_seed_6969_loss_function_NLLLoss_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0003_lr_decay_rate_0.25_unroll_steps_3_fusion_type_block'
MODEL_NAME = 'murel_dropout_corrected_wid_seed_1337_murel_attention_False_loss_function_NLLLoss_txt_enc_BayesianUniSkip_pooling_agg_max_pairwise_agg_max_batch_size_256_lr_0.0003_lr_decay_rate_0.25_unroll_steps_3_fusion_type_block'
MODEL_SAVE_NAME = OLD_TRAINED_FILES.format(MODEL_NAME)
best_ckpt = torch.load(MODEL_SAVE_NAME)
model.load_state_dict(best_ckpt['model'])
model.cuda()

/auto/homes/bat34/VQA_PartII/models/factory/ModelFactory.py:28: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


MurelNet(
  (murel_cell): MurelCell(
    (fusion_features): Block(
      (linear0): Linear(in_features=4800, out_features=1000, bias=True)
      (linear1): Linear(in_features=2048, out_features=1000, bias=True)
      (merge_linears0): ModuleList(
        (0): Linear(in_features=50, out_features=750, bias=True)
        (1): Linear(in_features=50, out_features=750, bias=True)
        (2): Linear(in_features=50, out_features=750, bias=True)
        (3): Linear(in_features=50, out_features=750, bias=True)
        (4): Linear(in_features=50, out_features=750, bias=True)
        (5): Linear(in_features=50, out_features=750, bias=True)
        (6): Linear(in_features=50, out_features=750, bias=True)
        (7): Linear(in_features=50, out_features=750, bias=True)
        (8): Linear(in_features=50, out_features=750, bias=True)
        (9): Linear(in_features=50, out_features=750, bias=True)
        (10): Linear(in_features=50, out_features=750, bias=True)
        (11): Linear(in_features=50, 

In [5]:
test_dataset = VQAv2Dataset(
        split="test",
        txt_enc=config['txt_enc'],
        bottom_up_features_dir=config['bottom_up_features_dir'],
        skipthoughts_dir=config['skipthoughts_dir'],
        processed_dir=config['processed_dir'],
        ROOT_DIR=config['ROOT_DIR'],
        vqa_dir=config['vqa_dir'])

test_loader = DataLoader(
        test_dataset,
        shuffle=True,
        batch_size=1,
        num_workers=0,
        collate_fn=test_dataset.collate_fn)

In [6]:
torch.cuda.empty_cache()

In [2]:
model_factory = ModelFactory()
config = model_factory.create_config('frcnn_concat')
test_dataset = VQAv2Dataset(
        split="test",
        txt_enc=config['txt_enc'],
        bottom_up_features_dir=config['bottom_up_features_dir'],
        skipthoughts_dir=config['skipthoughts_dir'],
        processed_dir=config['processed_dir'],
        ROOT_DIR=config['ROOT_DIR'],
        vqa_dir=config['vqa_dir'])

test_loader = DataLoader(
        test_dataset,
        shuffle=True,
        batch_size=1024,
        num_workers=config['num_workers'],
        collate_fn=test_dataset.collate_fn)

# Construct word vocabulary
word_vocabulary = [test_dataset.wid_to_word[i] for i in range(1, len(test_dataset.wid_to_word) + 1)]


model = model_factory.create_model('frcnn_concat', config, word_vocabulary)
model = model.cuda()


/auto/homes/bat34/VQA_PartII/models/factory/ModelFactory.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [3]:
OLD_TRAINED_FILES = '/local/scratch/bat34/baseline_trained_models/best_models/{}/best_model.pth'
#OLD_TRAINED_FILES = '/home/bat34/VQA_PartII/models/baseline/trained_models/best_models/{}/best_model.pth'

In [4]:
MODEL_NAME = 'agg_concat_min_corrected_wid_seed_2048_agg_type_min_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0003_fusion_type_concat_mlp'

In [5]:
MODEL_SAVE_NAME = OLD_TRAINED_FILES.format(MODEL_NAME)
best_ckpt = torch.load(MODEL_SAVE_NAME)
model.load_state_dict(best_ckpt['model'])

<All keys matched successfully>

In [6]:
def evaluate(config, model, val_loader,
                 evaluator, aid_to_ans,
                 RESULTS_FILE_PATH, device):
    model.eval()
    print('Running model on validation dataset..')
    with torch.no_grad():
        results = []
        total_batch_loss = 0
        batch_iter = 0
        for data in tqdm.tqdm(val_loader):
            item = {
                    'question_ids': data['question_ids'].cuda(),
                    'object_features_list': data['object_features_list'].cuda(),
                    'bounding_boxes': data['bounding_boxes'].cuda(),
                    'question_lengths': data['question_lengths'].cuda()
            }

            if 'murel' in config['name'] and config['use_graph_module']:
                item['graph_batch'] = data['graph'].to(device)

            inputs = item
            qids = data['question_unique_id']
            outputs = model(inputs)

            values, ans_indices = torch.max(outputs, dim=1)
            ans_indices = list(ans_indices)
            ans_indices = [tsr.item() for tsr in ans_indices]
            for qid, ans_idx in zip(qids, ans_indices):
                results.append({
                    'question_id': int(qid),
                    'answer': aid_to_ans[ans_idx]
                })
            batch_iter += 1

    print('Finished evaluating the model on the val dataset.')
    print('Saving results to %s' % RESULTS_FILE_PATH)
    with open(RESULTS_FILE_PATH, 'w') as f:
        json.dump(results, f)
    print('Done saving to %s' % RESULTS_FILE_PATH)
#     print('Calling VQA evaluation subroutine')
#     # We let the evaluator do all the tensorboard logging for accuracy
#     accuracy = evaluator.evaluate(RESULTS_FILE_PATH, 1)
#     print("Validation Results  Overall  accuracy: {:.2f}".format( accuracy))

In [7]:
evaluator = None
SAVE_FILE = '/home/bat34/VQA_PartII/models/'
SAVE_NAME = '/local/scratch/bat34/test_result_files/{}.json'.format(MODEL_NAME)
SAVE_NAME = SAVE_NAME if len(SAVE_NAME) <= 100 else SAVE_NAME[:100]
evaluate(config, model, test_loader, evaluator, test_dataset.aid_to_ans, 
            SAVE_NAME, 'cuda')

  0%|          | 0/438 [00:00<?, ?it/s]

Running model on validation dataset..


/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 438/438 [05:21<00:00,  1.36it/s]


Finished evaluating the model on the val dataset.
Saving results to /local/scratch/bat34/test_result_files/agg_concat_min_corrected_wid_seed_2048_agg_type_min_q_self_at
Done saving to /local/scratch/bat34/test_result_files/agg_concat_min_corrected_wid_seed_2048_agg_type_min_q_self_at
